In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You may also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm
import glob

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model.to(device)
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()
        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def generate(self, image_tensor, class_idx=None):
        image_tensor = image_tensor.to(device)
        self.model.eval()
        output = self.model(image_tensor)
        if class_idx is None:
            class_idx = output.argmax(dim=1).item()
        self.model.zero_grad()
        output[0, class_idx].backward()
        weights = self.gradients.mean(dim=[2, 3], keepdim=True)
        cam = (weights * self.activations).sum(dim=1, keepdim=True)
        cam = F.relu(cam)
        cam = F.interpolate(cam, size=image_tensor.shape[2:], mode='bilinear', align_corners=False)
        cam = cam.squeeze().cpu().numpy()
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        return cam

def apply_heatmap(image_tensor, cam):
    image_np = image_tensor.squeeze().permute(1, 2, 0).cpu().numpy()
    image_np = (image_np - image_np.min()) / (image_np.max() - image_np.min())
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    overlay = (0.4 * heatmap / 255.0 + 0.6 * image_np)
    return np.clip(overlay, 0, 1)

def focus_score(cam):
    return np.sum(cam) / cam.size

def attention_coverage(cam, threshold=0.5):
    return np.sum(cam > threshold) / cam.size

def intensity(cam):
    return np.mean(cam)

def calculate_independent_metrics(cam):
    return focus_score(cam), attention_coverage(cam), intensity(cam)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

real_paths = sorted(glob.glob('/kaggle/input/flowers-real/**/*.jpg', recursive=True))[:3000]
fake_paths = sorted(glob.glob('/kaggle/input/flower-102-real/**/*.jpg', recursive=True))[:3000]
assert len(real_paths) == len(fake_paths), "Mismatch in dataset length!"

model = models.resnet18(pretrained=True).to(device)
grad_cam = GradCAM(model, model.layer4[1].conv2)

real_metrics = []
fake_metrics = []

for idx, (real_img_path, fake_img_path) in enumerate(tqdm(zip(real_paths, fake_paths), total=len(real_paths))):
    try:
        real_img = Image.open(real_img_path).convert('RGB')
        fake_img = Image.open(fake_img_path).convert('RGB')
    except Exception as e:
        print(f"Error loading images at index {idx}: {e}")
        continue

    real_tensor = transform(real_img).unsqueeze(0).to(device)
    fake_tensor = transform(fake_img).unsqueeze(0).to(device)

    real_cam = grad_cam.generate(real_tensor)
    fake_cam = grad_cam.generate(fake_tensor)

    real_metrics.append(calculate_independent_metrics(real_cam))
    fake_metrics.append(calculate_independent_metrics(fake_cam))

    real_overlay = apply_heatmap(real_tensor, real_cam)
    fake_overlay = apply_heatmap(fake_tensor, fake_cam)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(real_overlay)
    axs[0].set_title("Real")
    axs[0].axis('off')
    axs[1].imshow(fake_overlay)
    axs[1].set_title("Fake")
    axs[1].axis('off')
    plt.tight_layout()
    plt.close()

real_metrics = np.array(real_metrics)
fake_metrics = np.array(fake_metrics)

print("\n=== 📊 Real Images (Average) ===")
print(f"Focus Score     : {real_metrics[:,0].mean():.4f}")
print(f"Attention Spread: {real_metrics[:,1].mean():.4f}")
print(f"Intensity       : {real_metrics[:,2].mean():.4f}")

print("\n=== 🧪 Fake Images (Average) ===")
print(f"Focus Score     : {fake_metrics[:,0].mean():.4f}")
print(f"Attention Spread: {fake_metrics[:,1].mean():.4f}")
print(f"Intensity       : {fake_metrics[:,2].mean():.4f}")


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 172MB/s]
  0%|          | 0/3000 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multipl


=== 📊 Real Images (Average) ===
Focus Score     : 0.4390
Attention Spread: 0.4117
Intensity       : 0.4390

=== 🧪 Fake Images (Average) ===
Focus Score     : 0.4361
Attention Spread: 0.4068
Intensity       : 0.4361
